In [2]:
import requests
import gzip
import pandas as pd
import shutil
import datetime

In [3]:
ftpstream = requests.get("https://smn.conagua.gob.mx/tools/GUI/webservices/?method=1", headers={'User-Agent': 'Mozilla/5.0'})

In [4]:
with open('./file.gz', 'wb') as file:
    file.write(ftpstream.content)

In [5]:
with gzip.open('./file.gz', 'rb') as f_in:
    with open('response.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [6]:
df = pd.read_json('./response.json')

In [7]:
df[['fecha', 'hora']] = df['dloc'].str.split('T', expand=True)
df['fecha'] = pd.to_datetime(df['fecha'])

In [8]:
df['hora'] = df['hora'].astype('int')

In [9]:
df

,cc,desciel,dh,dirvienc,dirvieng,dloc,ides,idmun,lat,lon,...,nes,nmun,prec,probprec,raf,tmax,tmin,velvien,fecha,hora
0,84.58,Cielo cubierto,6,Noreste,40.58,20230824T12,20,54,17.3884,-97.2290,...,Oaxaca,Magdalena Zahuatlán,0.60,95.0,21.83,20.72,13.36,12.09,2023-08-24,12
1,94.76,Cielo cubierto,6,Noreste,55.97,20230825T12,20,54,17.3884,-97.2290,...,Oaxaca,Magdalena Zahuatlán,1.59,95.0,16.47,21.09,14.75,7.82,2023-08-25,12
2,83.81,Cielo cubierto,6,Noreste,60.49,20230826T12,20,54,17.3884,-97.2290,...,Oaxaca,Magdalena Zahuatlán,6.43,95.0,13.32,22.25,15.51,7.47,2023-08-26,12
3,91.36,Cielo cubierto,6,Sureste,115.96,20230827T12,20,54,17.3884,-97.2290,...,Oaxaca,Magdalena Zahuatlán,12.74,95.0,13.58,20.85,15.27,5.75,2023-08-27,12
4,86.21,Cielo cubierto,6,Noreste,45.00,20230824T12,20,61,16.3672,-96.6368,...,Oaxaca,Monjas,7.63,95.0,18.66,25.62,17.34,8.71,2023-08-24,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847,27.77,Medio nublado,6,Este,97.65,20230827T12,19,45,25.9596,-100.2915,...,Nuevo León,Salinas Victoria,0.11,72.9,42.11,32.05,20.48,12.80,2023-08-27,12
9848,55.09,Cielo nublado,6,Sureste,114.57,20230824T12,19,46,25.7553,-100.2896,...,Nuevo León,San Nicolás de los Garza,0.00,0.0,36.40,34.22,25.79,14.20,2023-08-24,12
9849,19.52,Poco nuboso,6,Este,99.09,20230825T12,19,46,25.7553,-100.2896,...,Nuevo León,San Nicolás de los Garza,0.12,0.0,29.17,34.11,24.62,12.76,2023-08-25,12
9850,49.63,Medio nublado,6,Este,109.69,20230826T12,19,46,25.7553,-100.2896,...,Nuevo León,San Nicolás de los Garza,0.42,0.0,26.19,34.02,23.05,10.51,2023-08-26,12


In [10]:
df_2 = df[(df['fecha'] == datetime.date.today().isoformat())].groupby(['ides', 'idmun', 'nes', 'nmun'])[['tmax','tmin', 'prec']].mean()


In [11]:
df_2 = df_2.reset_index()

In [12]:
df_2

,ides,idmun,nes,nmun,tmax,tmin,prec
0,1,1,Aguascalientes,Aguascalientes,27.73,14.83,0.01
1,1,2,Aguascalientes,Asientos,27.02,13.87,0.06
2,1,3,Aguascalientes,Calvillo,26.93,15.77,0.05
3,1,4,Aguascalientes,Cosío,27.15,13.44,0.10
4,1,5,Aguascalientes,Jesús María,26.57,13.90,0.02
...,...,...,...,...,...,...,...
2458,32,54,Zacatecas,Villa Hidalgo,26.34,14.16,0.00
2459,32,55,Zacatecas,Villanueva,25.74,12.78,0.01
2460,32,56,Zacatecas,Zacatecas,27.68,11.69,0.00
2461,32,57,Zacatecas,Trancoso,28.60,14.52,0.00


# S3 integration

In [1]:
import boto3


